In [ ]:
# CELL 0 – Imports and physical constants

import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u   # ← THIS is what was missing
from scipy.interpolate import griddata

# Physical constants
G   = 6.67430e-11         # m^3 kg^-1 s^-2
c   = 2.99792458e8        # m/s
M_sun = 1.98847e30        # kg
kpc = 3.085677581e19      # m

# (optional) cosmology object if you need it later
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [ ]:
# ================================================================
# CELL 1 — Lens configuration input block for WFI 2033−4723
# ================================================================
# Change ONLY this cell for each lens.

# 1. Redshifts (lens & source)
# Values from COSMOGRAIL / H0LiCOW (e.g., Vuissoz+ 2008, Morgan+ 2004)
z_d = 0.661    # lens galaxy redshift
z_s = 1.662    # quasar source redshift

# 2. Angular-diameter distances using CELL 0 cosmology
D_d  = cosmo.angular_diameter_distance(z_d)            # [Mpc]
D_s  = cosmo.angular_diameter_distance(z_s)            # [Mpc]
D_ds = cosmo.angular_diameter_distance_z1z2(z_d, z_s)  # [Mpc]

# Convert to metres and build time-delay distance factor
D_d_m  = D_d.to(u.m).value
D_s_m  = D_s.to(u.m).value
D_ds_m = D_ds.to(u.m).value

time_delay_distance_factor = (1.0 + z_d) * (D_d_m * D_s_m / D_ds_m)
print(f"(1+z_d) D_d D_s / D_ds = {time_delay_distance_factor:.3e} m")

# 3. Einstein radius (arcsec)
# Typically around ~1.18–1.25" depending on model; 1.20" is a safe survey value.
theta_E_arcsec = 1.20

# 4. Lens mass ellipticity and orientation
# WFI2033 is moderately elliptical; PA from HST imaging.
axis_ratio_q      = 0.78       # b/a
position_angle_phi = -65.0     # degrees East of North

# 5. External shear (from H0LiCOW / Vuissoz+ 2008)
gamma_ext     = 0.10           # shear amplitude
phi_gamma_deg = -20.0          # shear PA (E of N)

# 6. External convergence (LOS mass-sheet)
# Environment is non-negligible; κ_env ≈ 0.05–0.10 in cosmography.
kappa_env = 0.08

# 7. Image positions (arcsec, lens-centered)
# HST-based astrometry (slightly rounded for the survey run)
# Images: A1, A2 (close pair), B, C
image_positions = {
    "A1": ( +0.737,  +0.533 ),
    "A2": ( +0.394,  +0.812 ),
    "B" : ( -0.720,  +0.493 ),
    "C" : ( +0.060,  -1.350 )
}

# 8. MGE parameters (shape only; normalization set by θ_E later)
# Generic elliptical early-type MGE (same structure as PG1115 / HE0435)
MGE_sigmas_arcsec = [
    0.06, 0.12, 0.20, 0.35, 0.55,
    0.80, 1.15, 1.70, 2.40, 3.80
]

MGE_amps = [
    7.0, 6.0, 4.8, 3.5, 2.4,
    1.6, 1.0, 0.6, 0.3, 0.12
]

(1+z_d) D_d D_s / D_ds = 1.519e+26 m


In [ ]:
# CELL 2 — Build grid & MGE model (does not change)

arcsec_to_rad = (np.pi/180)/3600

x_vals_arcsec = np.linspace(-5, 5, 500)
y_vals_arcsec = np.linspace(-5, 5, 500)
x_grid_arcsec, y_grid_arcsec = np.meshgrid(x_vals_arcsec, y_vals_arcsec)

def elliptical_gaussian(x, y, sigma, amp, q, phi_deg):
    phi = np.deg2rad(phi_deg)
    cosp, sinp = np.cos(phi), np.sin(phi)
    x_rot =  cosp*x + sinp*y
    y_rot = -sinp*x + cosp*y
    r2 = x_rot**2 + (y_rot/q)**2
    return amp*np.exp(-0.5*r2/sigma**2)

def psi_norm(x, y):
    psi = np.zeros_like(x)
    for sigma, amp in zip(MGE_sigmas_arcsec, MGE_amps):
        psi += elliptical_gaussian(
            x, y, sigma, amp,
            axis_ratio_q, position_angle_phi
        )
    return psi

psi_mass_norm = psi_norm(x_grid_arcsec, y_grid_arcsec)

In [ ]:
# CELL 3 — Normalize ψ via |∇ψ| = θ_E (does not change)

# Gradients
dpsi_dy_arcsec, dpsi_dx_arcsec = np.gradient(
    psi_mass_norm,
    y_vals_arcsec,
    x_vals_arcsec
)

# convert to radian deflection
dpsi_dx_rad = dpsi_dx_arcsec / arcsec_to_rad
dpsi_dy_rad = dpsi_dy_arcsec / arcsec_to_rad
alpha_mag = np.sqrt(dpsi_dx_rad**2 + dpsi_dy_rad**2)

# Einstein ring mask
theta_E_rad = theta_E_arcsec * arcsec_to_rad
r_grid = np.sqrt(x_grid_arcsec**2 + y_grid_arcsec**2)
ring_mask = np.abs(r_grid - theta_E_arcsec) <= 0.1

mean_defl_norm = alpha_mag[ring_mask].mean()
A_scale = theta_E_rad / mean_defl_norm

psi_mass_phys = psi_mass_norm * A_scale

In [ ]:
# CELL 4 — Add external shear (does not change)

phi_g = np.deg2rad(phi_gamma_deg)
cos2phi = np.cos(2*phi_g)
sin2phi = np.sin(2*phi_g)

x_rad = x_grid_arcsec * arcsec_to_rad
y_rad = y_grid_arcsec * arcsec_to_rad

psi_shear = 0.5*gamma_ext * (
    (x_rad**2 - y_rad**2)*cos2phi +
    2*x_rad*y_rad*sin2phi
)

psi_total = psi_mass_phys + psi_shear

In [ ]:
# ================================================================
# CELL 5 — Evaluate ψ_SFH at ALL image positions & compute all Δt
# ================================================================

def psi_at(theta_x, theta_y):
    """Interpolate ψ_SFH at an image position."""
    return griddata(
        (x_grid_arcsec.ravel(), y_grid_arcsec.ravel()),
        psi_total.ravel(),
        (theta_x, theta_y),
        method='cubic'
    )

# --- 1. Compute ψ for every image ---
psi_values = {}
for label, (x, y) in image_positions.items():
    psi_values[label] = psi_at(x, y)

print("ψ values at image positions:")
for k, v in psi_values.items():
    print(f"  ψ_{k} = {v:.6e}")
print()

# --- 2. Compute all pairwise delays ---
labels = list(image_positions.keys())
print("Pairwise SFH Time Delays:\n")

for i in range(len(labels)):
    for j in range(i+1, len(labels)):
        Li, Lj = labels[i], labels[j]
        psi_i = psi_values[Li]
        psi_j = psi_values[Lj]

        dpsi = psi_j - psi_i
        dt_raw = (dpsi * time_delay_distance_factor / c) / (24*3600)
        dt_corr = dt_raw / (1 - kappa_env)

        print(f"{Lj} – {Li}:")
        print(f"   Δψ = {dpsi:.6e}")
        print(f"   Raw SFH Δt = {dt_raw:.2f} days")
        print(f"   κ_env-corrected = {dt_corr:.2f} days\n")

ψ values at image positions:
  ψ_A1 = 2.637672e-11
  ψ_A2 = 2.934940e-11
  ψ_B = 3.655399e-11
  ψ_C = 1.811739e-11

Pairwise SFH Time Delays:

A2 – A1:
   Δψ = 2.972681e-12
   Raw SFH Δt = 17.43 days
   κ_env-corrected = 18.94 days

B – A1:
   Δψ = 1.017727e-11
   Raw SFH Δt = 59.67 days
   κ_env-corrected = 64.86 days

C – A1:
   Δψ = -8.259335e-12
   Raw SFH Δt = -48.42 days
   κ_env-corrected = -52.63 days

B – A2:
   Δψ = 7.204586e-12
   Raw SFH Δt = 42.24 days
   κ_env-corrected = 45.91 days

C – A2:
   Δψ = -1.123202e-11
   Raw SFH Δt = -65.85 days
   κ_env-corrected = -71.58 days

C – B:
   Δψ = -1.843660e-11
   Raw SFH Δt = -108.09 days
   κ_env-corrected = -117.49 days

